# 🎯 Static Proctoring Model Training

Train a frame-by-frame cheating detection model using LightGBM or XGBoost.

**Based on**: AutoOEP/Static/static_trainer.py

## Features Used
- Face verification status
- Eye gaze direction & iris position
- Head pose (x, y, z rotation)
- Mouth activity zone
- Prohibited objects detected (phone, earpiece, etc.)
- Hand distance from camera

In [ ]:
import os
import numpy as np
import pandas as pd
import joblib
import json
from datetime import datetime

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report
)

import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported")

## 1. Configuration

In [ ]:
# Paths
BASE_PATH = os.path.dirname(os.getcwd())  # ml/ directory
DATA_PATH = os.path.join(BASE_PATH, 'data', 'proctoring')
MODEL_PATH = os.path.join(BASE_PATH, 'models', 'proctoring')

# Create directories
os.makedirs(MODEL_PATH, exist_ok=True)

# Training settings
CONFIG = {
    'random_state': 42,
    'test_size': 0.2,
    'use_scaling': True,
    'model_type': 'lightgbm',  # 'lightgbm', 'xgboost', or 'ensemble'
    'threshold': 0.5,
}

# Feature columns (from AutoOEP)
FEATURE_COLUMNS = [
    'verification_result', 'num_faces', 'iris_pos', 'iris_ratio',
    'mouth_zone', 'mouth_area', 'x_rotation', 'y_rotation', 'z_rotation',
    'radial_distance', 'gaze_direction', 'gaze_zone',
    'watch', 'headphone', 'closedbook', 'earpiece', 'cell phone',
    'openbook', 'chits', 'sheet', 'H-Distance', 'F-Distance'
]

TARGET_COLUMN = 'is_cheating'

print(f"Model will be saved to: {MODEL_PATH}")
print(f"Using {len(FEATURE_COLUMNS)} features")

## 2. Load Data

In [ ]:
def load_data(data_path, feature_columns, target_column):
    """Load and prepare training data from CSV files."""
    
    # Look for CSV files
    csv_files = [f for f in os.listdir(data_path) if f.endswith('.csv')]
    
    if not csv_files:
        print(f"❌ No CSV files found in {data_path}")
        print("Please run feature extraction first or provide training data.")
        return None, None
    
    # Combine all CSVs
    dfs = []
    for f in csv_files:
        df = pd.read_csv(os.path.join(data_path, f))
        dfs.append(df)
        print(f"  Loaded {f}: {len(df)} rows")
    
    combined_df = pd.concat(dfs, ignore_index=True)
    print(f"\nTotal rows: {len(combined_df)}")
    
    # Check for required columns
    available_features = [c for c in feature_columns if c in combined_df.columns]
    missing_features = [c for c in feature_columns if c not in combined_df.columns]
    
    if missing_features:
        print(f"⚠️ Missing features: {missing_features}")
    
    if target_column not in combined_df.columns:
        print(f"❌ Target column '{target_column}' not found!")
        return None, None
    
    # Prepare X and y
    X = combined_df[available_features].copy()
    y = combined_df[target_column].copy()
    
    # Handle missing values
    X = X.fillna(0)
    
    print(f"\nFeatures shape: {X.shape}")
    print(f"Class distribution:\n{y.value_counts()}")
    
    return X, y

# Load data
X, y = load_data(DATA_PATH, FEATURE_COLUMNS, TARGET_COLUMN)

if X is not None:
    print("\n✅ Data loaded successfully")

## 3. Train/Test Split & Preprocessing

In [ ]:
if X is not None:
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=CONFIG['test_size'],
        random_state=CONFIG['random_state'],
        stratify=y
    )
    
    print(f"Training set: {len(X_train)} samples")
    print(f"Test set: {len(X_test)} samples")
    
    # Scale features
    scaler = None
    if CONFIG['use_scaling']:
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        print("✅ Features scaled")
    else:
        X_train_scaled = X_train.values
        X_test_scaled = X_test.values

## 4. Train Model

In [ ]:
def train_lightgbm(X_train, y_train, X_val, y_val):
    """Train a LightGBM classifier."""
    try:
        import lightgbm as lgb
    except ImportError:
        print("❌ LightGBM not installed. Run: pip install lightgbm")
        return None
    
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'learning_rate': 0.05,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': -1,
        'random_state': CONFIG['random_state']
    }
    
    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
    
    model = lgb.train(
        params,
        train_data,
        num_boost_round=500,
        valid_sets=[train_data, val_data],
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(period=100)
        ]
    )
    
    return model


def train_xgboost(X_train, y_train, X_val, y_val):
    """Train an XGBoost classifier."""
    try:
        import xgboost as xgb
    except ImportError:
        print("❌ XGBoost not installed. Run: pip install xgboost")
        return None
    
    model = xgb.XGBClassifier(
        objective='binary:logistic',
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        early_stopping_rounds=50,
        random_state=CONFIG['random_state']
    )
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=100
    )
    
    return model

In [ ]:
if X is not None:
    print(f"Training {CONFIG['model_type'].upper()} model...\n")
    
    if CONFIG['model_type'] == 'lightgbm':
        model = train_lightgbm(X_train_scaled, y_train, X_test_scaled, y_test)
    elif CONFIG['model_type'] == 'xgboost':
        model = train_xgboost(X_train_scaled, y_train, X_test_scaled, y_test)
    else:
        print(f"Unknown model type: {CONFIG['model_type']}")
        model = None
    
    if model:
        print("\n✅ Model trained successfully")

## 5. Evaluate Model

In [ ]:
def evaluate_model(model, X_test, y_test, threshold=0.5, model_type='lightgbm'):
    """Evaluate model and print metrics."""
    
    # Get predictions
    if model_type == 'lightgbm':
        y_proba = model.predict(X_test)
    else:
        y_proba = model.predict_proba(X_test)[:, 1]
    
    y_pred = (y_proba >= threshold).astype(int)
    
    # Metrics
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'auc_roc': roc_auc_score(y_test, y_proba)
    }
    
    print("=" * 50)
    print("MODEL EVALUATION RESULTS")
    print("=" * 50)
    for name, value in metrics.items():
        print(f"{name.upper():15}: {value:.4f}")
    
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=['Not Cheating', 'Cheating']))
    
    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Not Cheating', 'Cheating'],
                yticklabels=['Not Cheating', 'Cheating'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()
    
    return metrics, y_proba

if model is not None:
    metrics, y_proba = evaluate_model(
        model, X_test_scaled, y_test,
        threshold=CONFIG['threshold'],
        model_type=CONFIG['model_type']
    )

## 6. Save Model

In [ ]:
def save_model(model, scaler, feature_names, config, metrics, model_path):
    """Save model, scaler, and metadata."""
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Save model
    model_file = os.path.join(model_path, 'static_model.pkl')
    if config['model_type'] == 'lightgbm':
        model.save_model(model_file.replace('.pkl', '.txt'))
    else:
        joblib.dump(model, model_file)
    print(f"✅ Model saved: {model_file}")
    
    # Save scaler
    if scaler is not None:
        scaler_file = os.path.join(model_path, 'static_scaler.pkl')
        joblib.dump(scaler, scaler_file)
        print(f"✅ Scaler saved: {scaler_file}")
    
    # Save metadata
    metadata = {
        'model_type': config['model_type'],
        'feature_names': feature_names,
        'threshold': config['threshold'],
        'training_date': timestamp,
        'metrics': metrics
    }
    
    metadata_file = os.path.join(model_path, 'static_metadata.json')
    with open(metadata_file, 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"✅ Metadata saved: {metadata_file}")

if model is not None:
    save_model(model, scaler, list(X.columns), CONFIG, metrics, MODEL_PATH)
    print("\n🎉 Training complete! Models saved.")

## 7. Feature Importance

In [ ]:
if model is not None:
    # Get feature importance
    if CONFIG['model_type'] == 'lightgbm':
        importance = model.feature_importance(importance_type='gain')
    else:
        importance = model.feature_importances_
    
    # Create dataframe
    feature_importance = pd.DataFrame({
        'feature': list(X.columns),
        'importance': importance
    }).sort_values('importance', ascending=False)
    
    # Plot
    plt.figure(figsize=(10, 8))
    sns.barplot(data=feature_importance.head(15), x='importance', y='feature')
    plt.title('Top 15 Feature Importances')
    plt.xlabel('Importance')
    plt.tight_layout()
    plt.show()
    
    print("\nTop 10 Features:")
    print(feature_importance.head(10).to_string(index=False))